# Estimation from Samples

**CS5483 Data Warehousing and Data Mining**
___

In [ ]:
%reset -f
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# set figure size
plt.rcParams["figure.figsize"] = (8, 4)
# produce vector inline graphics
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

## Unbiased and Consistent Estimate

### Monte-Carlo Simulation

We will simulate the process of flipping a possibly biased coin using the module `numpy`, which is often renamed as `np` for convenience.  
See the [quickstart.](https://numpy.org/doc/stable/user/quickstart.html)

In [ ]:
import numpy as np
rng = np.random.default_rng(seed=0)

The above sets a random seed so that the result is reproducible.

The following chooses the probability $p$ of head. It uniformly randomly pick the value from the unit interval $[0,1)$.

In [ ]:
p = rng.random()  # randomly generate the probability of head coming up.

To generate a sequence of random coin tosses:

In [ ]:
size = 5000
coin_tosses = rng.choice(['H', 'T'], size=size, p=[p, 1 - p])
coin_tosses

The characters `'H'` and `'T'` denotes head and tail respectively.

### M-Estimation

We can estimate $p$ by sample average also called an M-estimate

$$
\hat{p} := \frac1n \sum_{i=0}^{n-1} z_i,
$$

where $z_1,\dots,z_n$ are i.i.d. samples of

$$
Z := \begin{cases}
1 & \text{if head comes up}\\
0 & \text{if tail comes up.}
\end{cases}
$$

$\hat{p}$, also called the empirical probability, is simply the fraction of heads coming up in $n$ coin tosses:

In [ ]:
(coin_tosses == 'H').mean()

**Exercise** The following proves that $\hat{p}$ is an unbiased estimate:

$$
\begin{aligned}
E[\hat{p}] &= E\left[\frac1n \sum_{i=0}^{n-1} z_i \right]\\
&= \frac1n \sum_{i=0}^{n-1} \underbrace{E[z_i]}_{\smash{=p}} && \text{by ___}\\
&= p.
\end{aligned}
$$

Fill in the missing reasoning.

YOUR ANSWER HERE

**Exercise** The following shows that $\hat{p}$ is a consistent estimate:

$$
\begin{aligned}\sigma &:=\sqrt{\operatorname{Var}[\hat{p}]}\\
&= \sqrt{\operatorname{Var}\left[\frac1{n} \sum_{i=1}^n z_i\right]}\\
&= \sqrt{\frac1{n^2} \operatorname{Var}\left[\sum_{i=1}^n z_i\right]}\\
&=\sqrt{\frac1{n^2} \sum_{i=1}^n \underbrace{\operatorname{Var}[z_i]}_{\smash{=p(1-p)}}} && \text{by ___}\\
&=\sqrt{\frac{p(1-p)}{n}}, \end{aligned}
$$

which goes to $0$ as $n$ goes to $\infty$.

Fill in the missing reasoning.

YOUR ANSWER HERE

By the [Central Limit Theorem](https://en.wikipedia.org/wiki/Central_limit_theorem) as $n$ goes to $\infty$, the estimate has a gaussian distribution:

<a title="M. W. Toews, CC BY 2.5 &lt;https://creativecommons.org/licenses/by/2.5&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Standard_deviation_diagram.svg"><img alt="Standard deviation diagram" src="https://upload.wikimedia.org/wikipedia/commons/8/8c/Standard_deviation_diagram.svg"></a>

Over $95\%$ of the time, the estimator falls within $2$ standard deviation away from the ground truth, i.e.,

$$
\Pr(\hat{p}\in [p-2\sigma, p+2\sigma]) \geq 0.95
$$

To illustrate the above qualities of the estimator by plotting the estimate $\hat{p}$ for different sample size $n$.

In [ ]:
n = np.arange(1, size + 1)
phat = (coin_tosses == 'H').cumsum() / n  # use first n tosses to estimate
sigma = (p * (1 - p) / n)**0.5  # standard deviations of the estimates

A powerful module for plotting graphs is `matplotlib.pyplot`, which is often renamed to `plt` for convenience.  
See the [quickstart.](https://matplotlib.org/tutorials/introductory/pyplot.html)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
# set figure size
plt.rcParams["figure.figsize"] = (8, 4)
# produce vector inline graphics
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

We have additionally set the plots to use vector formats so they have unlimited resolution.

In [ ]:
f, ax = plt.subplots()  # create the figure and axes

# plot the ground truth p
ax.axhline(p, color='red')

# fill the region 2 sigma away from p
ax.fill_between(n,
                p - 2 * sigma,
                p + 2 * sigma,
                color='red',
                alpha=0.2,
                label=r'$p\pm 2\sigma$')

# plot the estimates phat
ax.plot(n, phat, marker='.', color='blue', linestyle='', markersize=1)

# configure the plot
ax.set_ylim([0, 1])
ax.set_xlim([0, n.size])
ax.set_title(r'Plot of $\hat{p}$ vs sample size')
ax.set_xlabel('sample size')
ax.set_ylabel('probability')
ax.legend()
f.show()

### Coin-tossing game

Imagine playing the following coin-tossing game:

- You win if a coin toss comes up head.
- You get to choose 1 out of the 2 coins $i\in \{0,1\}$ with unknown probability $p_i$ of coming up head.
- You can flip each coin $n$ times before making the choice.

A particular strategy to play the game is to 
1. estimate the chance $p_i$ of winning by the empirical probability $\hat{p}_i$ for each coin $i\in \{0,1\}$, and
1. select the coin with larger $\hat{p}_i$ to toss.

**Exercise** With the above strategy:
1. Is the chance of winning $</=/> \max_i p_i$?
2. Is the chance of winning $</=/> E[\max_i \hat{p}_i]$?
3. Is the strategy the optimal winning strategy?

YOUR ANSWER HERE

### Simulate the strategy

You can check some of your answers by simulating the strategy.

First create a list of probabilities of head for `m=2` coins. We will set the probability to 0.5, but you can rerun the notebook later with other choices probabilities. You can also increase `m`.

In [ ]:
m = 2
p = np.ones(m, dtype=float) * 0.5
# To generate the probability randomly instead, use
# p = rng.random(m)
p.shape

Instead of generating a sequence of coin toss, we will simulate $\hat{p}_i$ directly using the binomial distribution as $n\hat{p}_i\sim \operatorname{Binomial}(n,p_i)$

In [ ]:
size = 5000
n = np.arange(1,size+1)
k = 1000
phat = rng.binomial(n[:,None,None],p[None,:,None],(size,m,k)) / n[:,None,None]
max_phat = phat.max(axis=1)
max_phat.shape

`max_phat` is a 2-dimensional array samples of $\max_{i}\hat{p}_i$:
- The first axis indexes samples obtained from different number of tosses.
- The second axis indexes `k` independent samples for the same number of tosses.

The `k` independent samples can be used to give estimates of $E[\max_{i}\hat{p}_i]$ as follows.

In [ ]:
# plot the maximum of the probability estimates of each coin
plt.plot(n,max_phat.mean(axis=-1), linestyle='',marker='.',color='blue',markersize=1)

# plot the maximum of true probabilities of head
plt.axhline(p.max(),color='red',label=r'$\max_i p_i$')

plt.ylim([0,1])
plt.xlim([n[0],n[-1]])
plt.title(r'Plot of $E[\max_i\hat{p}_i]$ vs sample size')
plt.xlabel('sample size')
plt.ylabel('probability')
plt.legend()
plt.show()

**Exercise** From the above plot: 
1. How to see whether $\max_{i}\hat{p}_i$ is an overly-optimistic estimate of the chance of winning from the above plot? 
2. If the blue curve converge to the red curve as the sample size increases, does it necessarily mean that the estimate is consistent?

YOUR ANSWER HERE